In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import matplotlib.pyplot as plt # for plotting
import numpy as np # for transformation
import pandas as pd # to deal with dataframes
import os 

import torch # PyTorch package
import torchvision # load datasets
import torchvision.transforms as transforms # transform data
import torch.nn as nn # basic building block for neural networks
import torch.nn.functional as F # import convolution functions like Relu
import torch.optim as optim # optimzer
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader # For making datasets
from PIL import Image # For opening images from given path
from sklearn.metrics import accuracy_score,f1_score



In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
num_epochs = 10 # Please set this according to your training time

In [5]:
! unzip "/kaggle/input/noaa-right-whale-recognition/imgs.zip" -d "/kaggle/working"

In [6]:
df = pd.read_csv('/kaggle/input/noaa-right-whale-recognition/train.csv')

from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
from sklearn.model_selection import train_test_split

df['label'] = labelencoder.fit_transform(df['whaleID'])
assert len(df['whaleID'].unique()) == len(df['label'].unique())
num_labels = len(df['whaleID'].unique())
print(num_labels)

X = np.array(df['Image'])
Y = np.array(df['label'])

x_train,x_val,y_train,y_val = train_test_split(X,Y,test_size=0.2)

train_df, val_df = pd.DataFrame(), pd.DataFrame()
train_df['Image'], val_df['Image'] = x_train, x_val
train_df['label'], val_df['label'] = y_train, y_val

In [7]:
print(df.head(2))
print()
print(x_train.shape,x_val.shape)
print()
print(train_df.head(2))
print()
print(val_df.head(2))
print()

In [8]:

class WhaleDataset(Dataset):

    def __init__(self, data_frame, root_dir, transform=None):
        self.data_frame = data_frame
        self.root_dir = root_dir
        self.transform = transform   # image transform
        self.images = []
        self.labels = []

        for i in range(self.data_frame.shape[0]):
            img_name = os.path.join(self.root_dir, self.data_frame.iloc[i, 0])
            if not os.path.exists(img_name):
                print(img_name)
                continue
            self.labels.append(self.data_frame.iloc[i, -1])
            temp_image = Image.open(img_name)
            self.images.append(self.transform(temp_image))

    def __len__(self):
        # Return the length of the dataset
        return len(self.images)
    
    def __getitem__(self, idx):
        return (self.images[idx], self.labels[idx])

In [9]:
# typical image transformation
transform = transforms.Compose([
            torchvision.transforms.Resize(256),
            torchvision.transforms.CenterCrop(224),
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize(
                mean = (0.485,0.456,0.406),
                std = (0.229,0.224,0.225))]
)

In [11]:
trainset = WhaleDataset(
    data_frame = train_df.iloc[:20,:],
    root_dir = '/kaggle/working/imgs',
    transform=transform)
valset = WhaleDataset(
    data_frame = val_df,
    root_dir = '/kaggle/working/imgs',
    transform=transform)


In [12]:
# set batch_size
batch_size = 16

# load train data
traindataloader = DataLoader(trainset, batch_size=batch_size,
                            shuffle=True)

# load test data
valdataloader = DataLoader(valset, batch_size=64,
                         shuffle=False)

In [13]:
class ConvNet(nn.Module):

    ''' Models a simple Convolutional Neural Network '''
	
    def __init__(self):
        ''' initialize the network '''
        super(ConvNet, self).__init__()
#         self.resnet = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
        self.resnet = models.resnet34(pretrained=True)
        
        for param in self.resnet.parameters():
            param.requires_grad = False
        
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Sequential(nn.Linear(num_ftrs, 447))
        self.logsoftmax = nn.LogSoftmax(dim = 1) # Since it is multi-class classification we used Softmax/LogSoftmax

    def forward(self, x):
        ''' the forward propagation algorithm '''
        x = self.resnet(x)
        x = self.logsoftmax(x)
        return x

net = ConvNet()
net.to(device)
# print(net)
# for param in net.parameters():
#     if param.requires_grad:
#         print(param.shape)

In [14]:
criterion = nn.NLLLoss()  # using NLL Loss
optimizer = optim.Adam(net.parameters(), lr=0.01) # Adam Optimiser

for epoch in range(50):  # loop over the dataset multiple times

    for i, data in enumerate(traindataloader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # print statistics
        if i % 100 == 0:   
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, loss.item()))

In [15]:
# Evaluation mode
net.eval()
correct = 0
total = 0
all_labels = []
all_predicted = []
all_outputs = []
with torch.no_grad():
    for data in valdataloader:
        images, labels = data
        images = images.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        predicted = predicted.to('cpu')
        outputs = outputs.to('cpu')
        all_labels.append(labels)
        all_predicted.append(predicted)
        all_outputs.append(outputs)
    a1,a2,a3 = all_labels,all_predicted,all_outputs
    a1 = torch.cat(a1)
    a2 = torch.cat(a2)
    a3 = torch.cat(a3)
    print(f'F1 score is {np.mean(f1_score(a1,a2,average = None))}') # Printing F1 score
    print(f'Accuracy score is {np.mean(accuracy_score(a1,a2))}') # Printing accuracy_score


In [72]:
a3.shape

In [73]:
test_data = pd.read_csv('../input/noaa-right-whale-recognition/sample_submission.csv')
label = list(test_data)

In [74]:
len(label)

In [75]:
indices = a2.numpy()
indices.shape

In [76]:
predictions = a3.numpy()
predictions.shape

In [77]:
for i in range(indices.shape[0]):
    for j in range(predictions.shape[1]):
        if predictions[i][j] < predictions[i,indices[i]]:
            predictions[i][j] = 0
        else:
            predictions[i][j] = 1

In [78]:
for i in range(447):
    test_data[label[i+1]] = pd.Series(predictions[:,i])

In [79]:
test_data.fillna(value = 0,inplace = True)

In [80]:
saveFileName = '/kaggle/working/submission.csv'
submission = test_data
submission.to_csv(saveFileName, index=False)

In [81]:
submission